#### Ensure yfinance package is installed.

In [1]:
# Install yfinance package.
!pip install yfinance html5lib --q

#### Define function to import metrics from a given stock list.

In [2]:
import yfinance as yf
import pandas as pd

def get_stock_financial_metrics(ticker_symbol):
    """
    Retrieves key financial metrics for a given stock ticker using yfinance.

    Args:
        ticker_symbol (str): The stock ticker symbol (e.g., "AAPL", "MSFT").

    Returns:
        dict: A dictionary containing the financial metrics.
              Returns None for metrics not available.
              Returns an error message string if the ticker is invalid or data cannot be fetched.
    """
    try:
        stock = yf.Ticker(ticker_symbol)
        info = stock.info

        # A more robust check for valid ticker data
        if not info or 'symbol' not in info or info.get('symbol', '').lower() != ticker_symbol.lower():
            # Check if it's a known "bad" ticker pattern from yfinance for delisted/problematic ones
            if info.get('regularMarketPrice') is None and info.get('logo_url') == '': # Common pattern for invalid tickers
                 return f"Could not retrieve valid data for ticker: {ticker_symbol}. It might be an invalid or delisted ticker."
            # If 'symbol' is present but doesn't match, it's odd, but let's flag it.
            # If 'symbol' is missing, it's definitely problematic.
            if 'symbol' not in info:
                return f"Could not retrieve valid data for ticker: {ticker_symbol}. Essential 'symbol' info missing."


        metrics = {
            "ticker": ticker_symbol, # Ensure ticker is always present
            "price": info.get('currentPrice', info.get('regularMarketPrice', info.get('previousClose'))),
            "pe_ratio": info.get('trailingPE', info.get('forwardPE')),
            "eps": info.get('trailingEps', info.get('forwardEps')),
            "roe": info.get('returnOnEquity'),
            "roa": info.get('returnOnAssets'),
            "profit_margin": info.get('profitMargins'), # Added profit margin
            "book_value_per_share": info.get('bookValue'),
            "shares_outstanding": info.get('sharesOutstanding'),
            "price_to_book": info.get('priceToBook'),
            "shortName": info.get('shortName') # Adding company name for clarity
        }
        return metrics

    except Exception as e:
        # For truly problematic tickers, yfinance might raise an exception before .info
        # or if .info itself is problematic (e.g., not a dict)
        return {
            "ticker": ticker_symbol,
            "price": None,
            "pe_ratio": None,
            "eps": None,
            "roe": None,
            "roa": None,
            "profit_margin": None, # Added profit margin
            "book_value_per_share": None,
            "shares_outstanding": None,
            "price_to_book": None,
            "shortName": f"Error: {str(e)}", # Store error in a field
            "error_message": str(e) # Explicit error message field
        }


def get_financials_for_stock_list(ticker_list):
    """
    Fetches financial metrics for a list of stock tickers and returns them as a Pandas DataFrame.

    Args:
        ticker_list (list): A list of stock ticker symbols (e.g., ["AAPL", "MSFT", "GOOGL"]).

    Returns:
        pandas.DataFrame: A DataFrame containing the financial metrics for each stock.
                          Includes an 'error_message' column for tickers where data couldn't be fetched.
    """
    all_metrics_data = []
    for ticker in ticker_list:
        print(f"Fetching data for {ticker}...")
        data = get_stock_financial_metrics(ticker)
        
        # If the function returns a string (our old error handling), convert to dict
        if isinstance(data, str) and "Could not retrieve" in data: # Check for our specific error string
            metrics_dict = {
                "ticker": ticker, "price": None, "pe_ratio": None, "eps": None,
                "roe": None, "roa": None, "profit_margin": None, # Added profit margin
                "book_value_per_share": None, "shares_outstanding": None, 
                "price_to_book": None, "shortName": None,
                "error_message": data
            }
        elif isinstance(data, dict):
            metrics_dict = data
            if "error_message" not in metrics_dict: # Ensure error_message field exists
                 metrics_dict["error_message"] = None
        else: # Should not happen with current get_stock_financial_metrics
            metrics_dict = {
                "ticker": ticker, "price": None, "pe_ratio": None, "eps": None,
                "roe": None, "roa": None, "profit_margin": None, # Added profit margin
                "book_value_per_share": None, "shares_outstanding": None, 
                "price_to_book": None, "shortName": None,
                "error_message": "Unknown error structure from get_stock_financial_metrics"
            }
            
        all_metrics_data.append(metrics_dict)
        
    # Create DataFrame from the list of dictionaries
    df = pd.DataFrame(all_metrics_data)
    
    # Reorder columns to have ticker and shortName first, and error_message last
    if not df.empty:
        cols = ["ticker", "shortName", "price", "pe_ratio", "eps", "roe", "roa", "profit_margin",
                "book_value_per_share", "shares_outstanding", "price_to_book", "error_message"]
        # Filter out columns not present in the DataFrame (e.g., if all tickers failed identically)
        existing_cols = [col for col in cols if col in df.columns]
        df = df[existing_cols]
        
    return df

#### Execute function with current stock list and store into df "successful_data_df"

In [ ]:
# List of stock tickers you want to analyze
stock_list = ["APPL"]

In [9]:
# Execute conditional for each stock and return specified metrics.
if __name__ == "__main__":
    print("Starting financial data retrieval...")
    financials_df = get_financials_for_stock_list(stock_list)

    #print("\n--- Financial Data DataFrame ---")
    #print(financials_df)

    # Further analysis or saving the DataFrame
    if not financials_df.empty:
        #print("\n--- DataFrame Info ---")
        #financials_df.info()

        # Example: Filter out rows with errors for cleaner analysis
        successful_data_df = financials_df[financials_df['error_message'].isnull()].copy() # Use .copy() to avoid SettingWithCopyWarning
        
        # Convert relevant columns to numeric, coercing errors to NaN
        numeric_cols = ["price", "pe_ratio", "eps", "roe", "roa", "profit_margin", 
                        "book_value_per_share", "shares_outstanding", "price_to_book"]
        for col in numeric_cols:
            if col in successful_data_df.columns:
                successful_data_df[col] = pd.to_numeric(successful_data_df[col], errors='coerce')

Starting financial data retrieval...
Fetching data for APPL...


---

#### Analysis: Explore DF

In [10]:
successful_data_df

,ticker,shortName,price,pe_ratio,eps,roe,roa,profit_margin,book_value_per_share,shares_outstanding,price_to_book,error_message
0,APPL,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
